In [1]:
#imports from Sanzid
import h5py
import json
import matplotlib.pyplot as plt
import numpy as np
import numpy
import statistics
from numpy import loadtxt
import matplotlib.pyplot as plt
import pandas
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statistics import stdev
import math
import h5py

import numpy as np
import time

from scipy.signal import butter,filtfilt
import sys
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph
import seaborn as sns # used for plot interactive graph.
import pandas
import matplotlib.pyplot as plt

# from tsf.model import TransformerForecaster


# from tensorflow.keras.utils import np_utils
import itertools
###  Library for attention layers
import pandas as pd
import os
import numpy as np
#from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split

import statistics
import gc
import torch.nn.init as init

############################################################################################################################################################################
############################################################################################################################################################################

import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.utils.weight_norm as weight_norm
from sklearn.preprocessing import StandardScaler


import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
#from torchsummary import summary
from torch.nn.parameter import Parameter

#from torchsummary import summary



import torch.optim as optim
import gc

#from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler



#imports for this notebook using sensor distillation utils
import sys #you need this to add path to utils folder
sys.path.append('../Sensor_distillation_utils') #now you can import from utils folder
from sensor_distillation import train_SD
from student_model import student
from teacher_model import teacher
from train import train_kinematics
from predictions import prediction_test

#loading in data from lightweight kinematics net
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import os

In [2]:

# Convert an array of values into a dataset matrix
def create_dataset_present(dataset_1, window=100):
    # Ensure the dataset is a NumPy array
    if isinstance(dataset_1, pd.DataFrame):
        dataset_1 = dataset_1.values
    dataX = []
    k = 0
    shift = 50
    for i in range(int(len(dataset_1) / shift) - 1):
        j = shift * k
        a = dataset_1[j:j + window, :]
        dataX.append(a)
        k = k + 1
    return np.array(dataX)
"""""
#WINDOWING DATA
## Creating a dataset with overlapping window of 100 samples with overlap of 50 samples ##

# # convert an array of values into a dataset matrix
def create_dataset_present(dataset_1, window=100):
  dataX= []
  k=0
  shift=50
  for i in range(int(len(dataset_1)/shift)-1):
    j=shift*k
    a = dataset_1[j:j+window,:]
    dataX.append(a)
    k=k+1
  return np.array(dataX)
"""""

'""\n#WINDOWING DATA\n## Creating a dataset with overlapping window of 100 samples with overlap of 50 samples ##\n\n# # convert an array of values into a dataset matrix\ndef create_dataset_present(dataset_1, window=100):\n  dataX= []\n  k=0\n  shift=50\n  for i in range(int(len(dataset_1)/shift)-1):\n    j=shift*k\n    a = dataset_1[j:j+window,:]\n    dataX.append(a)\n    k=k+1\n  return np.array(dataX)\n'

In [3]:


# Load and clean the combined CSV data
import sys #you need this to add path to utils folder
sys.path.append('../../datacollection') #now you can import from utils folder
from batch_process import combine_and_save_data

batch_size = 64

remake_combined = False
if remake_combined == True:   
    combine_and_save_data("../../datacollection/vicon/subject_1/processed", "../../datacollection/vicon/subject_1/sensors", "../../datacollection/vicon/subject_1/combined")

combined_csv = pd.read_csv("../../datacollection/vicon/subject_1/combined/P001_T001_armSwing_fast_combined.csv")

print(combined_csv.shape)

train_joint_data = combined_csv.iloc[:, 25:29]
columns_to_drop = ["arm_rot_r"]
train_joint_data = train_joint_data.drop(columns = columns_to_drop)
print(train_joint_data.columns)
train_imu_data = combined_csv.iloc[:, 49:]
print(train_imu_data.shape)
#50 parriba
""""
# Get two 3-channel arrays for ACC and GYRO for sensor 2
data_acc = combined_csv[['ACCX2', 'ACCY2', 'ACCZ2']].values
data_gyr = combined_csv[['GYROX2', 'GYROY2', 'GYROZ2']].values

# Normalize the data
scaler_acc = StandardScaler()
scaler_gyr = StandardScaler()
scaler_joints = StandardScaler()

data_acc = scaler_acc.fit_transform(data_acc)
data_gyr = scaler_gyr.fit_transform(data_gyr)
data_joints = combined_csv[['elbow_flex_r']].values
data_joints = scaler_joints.fit_transform(data_joints)

print (data_acc.shape, data_gyr.shape, data_joints.shape)

#data_acc = train X data_gyr = train X data_joints = train y
################
w=100

train_X_3_acc=create_dataset_present(data_acc,w)
train_X_3_gyr=create_dataset_present(data_gyr,w)
train_y_3=create_dataset_present(data_joints,w)

train_y_3=train_y_3.reshape(train_y_3.shape[0],w,6)

train_X_1D, X_validation_1D, train_y_5, Y_validation = train_test_split(train_X_3,train_y_3, test_size=0.20, random_state=True)

print(train_X_1D.shape,train_y_5.shape,X_validation_1D.shape,Y_validation.shape)

features=6
train_X_2D=train_X_1D.reshape(train_X_1D.shape[0],train_X_1D.shape[1],features,8)
X_validation_2D= X_validation_1D.reshape(X_validation_1D.shape[0], X_validation_1D.shape[1],features,8)

# Create dataset instances with overlapping windows
dataset = Sensor_Distillation_Dataset(data_acc, data_gyr, data_joints, w=1000, overlap=500)


# Split the dataset into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print (len(train_dataset), len(val_dataset))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(len(val_loader))
"""""

(64581, 97)
Index(['arm_flex_r', 'arm_add_r', 'elbow_flex_r'], dtype='object')
(64581, 48)


'"\n# Get two 3-channel arrays for ACC and GYRO for sensor 2\ndata_acc = combined_csv[[\'ACCX2\', \'ACCY2\', \'ACCZ2\']].values\ndata_gyr = combined_csv[[\'GYROX2\', \'GYROY2\', \'GYROZ2\']].values\n\n# Normalize the data\nscaler_acc = StandardScaler()\nscaler_gyr = StandardScaler()\nscaler_joints = StandardScaler()\n\ndata_acc = scaler_acc.fit_transform(data_acc)\ndata_gyr = scaler_gyr.fit_transform(data_gyr)\ndata_joints = combined_csv[[\'elbow_flex_r\']].values\ndata_joints = scaler_joints.fit_transform(data_joints)\n\nprint (data_acc.shape, data_gyr.shape, data_joints.shape)\n\n#data_acc = train X data_gyr = train X data_joints = train y\n################\nw=100\n\ntrain_X_3_acc=create_dataset_present(data_acc,w)\ntrain_X_3_gyr=create_dataset_present(data_gyr,w)\ntrain_y_3=create_dataset_present(data_joints,w)\n\ntrain_y_3=train_y_3.reshape(train_y_3.shape[0],w,6)\n\ntrain_X_1D, X_validation_1D, train_y_5, Y_validation = train_test_split(train_X_3,train_y_3, test_size=0.20, random_

In [4]:
#load in test data
remake_combined_test = False
if remake_combined_test == True:   
    combine_and_save_data("../../datacollection/vicon/subject_2/processed", "../../datacollection/vicon/subject_2/sensors", "../../datacollection/vicon/subject_2/combined")

combined_csv_test = pd.read_csv("../../datacollection/vicon/subject_2/combined/P002_T001_armSwing_fast_combined.csv")


test_joint_data = combined_csv.iloc[:, 25:29]
columns_to_drop = ["arm_rot_r"]
test_joint_data = test_joint_data.drop(columns = columns_to_drop)
print(test_joint_data.shape)
test_imu_data = combined_csv.iloc[:, 49:]
print(test_imu_data.shape)


(64581, 3)
(64581, 48)


In [5]:
#WINDOW DATA
### Reconstruction/Present Dataset ###
w=100

train_X_3=create_dataset_present(train_imu_data,w)
train_y_3=create_dataset_present(train_joint_data,w)
print (train_y_3.shape)
test_X_1D=create_dataset_present(test_imu_data,w)
test_y=create_dataset_present(test_joint_data,w)

(1290, 100, 3)


In [6]:
#RESHAPING TARGETS TO 3D ARRAY 
# -rows, -window_length, -columns
train_y_3=train_y_3.reshape(train_y_3.shape[0],w,3)
test_y=test_y.reshape(test_y.shape[0],w,3)

In [7]:
#SPLIT TRAINING & VALIDATION
train_X_1D, X_validation_1D, train_y_5, Y_validation = train_test_split(train_X_3,train_y_3, test_size=0.20, random_state=True)

print(train_X_1D.shape,train_y_5.shape,X_validation_1D.shape,Y_validation.shape)
#shape is num_rows, window_length, num_channels

(1032, 100, 48) (1032, 100, 3) (258, 100, 48) (258, 100, 3)


In [8]:
#features or columns in the dataset
channels=6
#1D is -rows, -window_length, columns
#2D is just reshaping the 1D to -rows, -window_length, -columns, -num_sensors
train_X_2D=train_X_1D.reshape(train_X_1D.shape[0],train_X_1D.shape[1],channels,8)
test_X_2D=test_X_1D.reshape(test_X_1D.shape[0],test_X_1D.shape[1],channels,8)
X_validation_2D= X_validation_1D.reshape(X_validation_1D.shape[0],
                                                   X_validation_1D.shape[1],channels,8)


print(train_X_2D.shape,test_X_2D.shape,X_validation_2D.shape)

(1032, 100, 6, 8) (1290, 100, 6, 8) (258, 100, 6, 8)


In [9]:
#TURNING BACK THE DATA TO DESIRED FORM
## Evaluting data with the original form without any overlaps

# # convert an array of values into a dataset matrix
def unpack_dataset_present(dataset_1):
  dataX= []
  k=1
  l=0
  shift=100
  for i in range(int(len(dataset_1)/shift)-1):
    j=shift*k
    a = dataset_1[l:j,:]
    l=0
    l=j+50
    dataX=np.append(dataX,a)
    k=k+1
    j=0
  return np.array(dataX)

In [10]:
#PREP THE DATA
### IMUs- Chest, Waist, Right Foot, Right shank, Right thigh, Left Foot, Left shank, Left thigh, 2D-body coordinate
### 0:48- IMU, 48:92-2D body coordinate, 92:97-- Target


### Data Processing

batch_size = 64

#MAKE 1D INTO TENSOR
train_features_1D = torch.Tensor(train_X_1D)
val_features_1D = torch.Tensor(X_validation_1D)
test_features_1D = torch.Tensor(test_X_1D)

#MAKE 2D INTO TENSOR
train_features_2D = torch.Tensor(train_X_2D)
val_features_2D = torch.Tensor(X_validation_2D)
test_features_2D = torch.Tensor(test_X_2D)

#TARGETS TO TENSORS
train_targets = torch.Tensor(train_y_5)
val_targets = torch.Tensor(Y_validation)
test_targets = torch.Tensor(test_y)

print(test_targets.shape)

## all Modality Features
train = TensorDataset(train_features_1D, train_features_2D, train_targets)
val = TensorDataset(val_features_1D, val_features_2D,val_targets)
test = TensorDataset(test_features_1D,test_features_2D,test_targets)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=False)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=True, drop_last=False)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=False)


torch.Size([1290, 100, 3])


In [11]:

#define device
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [12]:
#SOME MODEL PARAMETERS
alpha=0.50
num_epoch=1

In [13]:
k_1=0
k_2=48
k_3=0
k_4=8

lr = 0.001
model = teacher(k_2-k_1,k_4-k_3,100)

gait_Net_teacher = train_kinematics(device, train_loader, val_loader, lr,num_epoch,model,"../../datacollection/vicon/subject_1/test/test.pth",k_1, k_2, k_3, k_4)

data_features_1D shape: torch.Size([64, 100, 48])
data_features_2D shape: torch.Size([64, 100, 6, 8])
data_targets shape: torch.Size([64, 100, 3])
##################################
first passage through encoder gru
input to encoder_gru, torch.Size([64, 100, 48])
shape of out_1 after lstm1, torch.Size([64, 100, 256])
shape of out_1 dropout1, torch.Size([64, 100, 256])
shape of out_2 after lstm1, torch.Size([64, 100, 128])
shape of out_2 dropout2, torch.Size([64, 100, 128])
shape of out_2 after flatten, torch.Size([64, 12800])
necessary shape for last linear layer = 128000, 600
shape encoder_gru output, torch.Size([64, 300])
shape encoder_gru output after view, torch.Size([64, 100, 3])
##################################
forward pass encoder 1D
shape of input, torch.Size([64, 100, 48])
input_1D_N_1: torch.Size([64, 48, 100])
input_1D_N_1: torch.Size([64, 48, 100])
input_1D_N_1: torch.Size([64, 48, 100])
Shape after transpose: torch.Size([64, 48, 100])
Shape after conv1 relu: torch.Size([

In [14]:
gait_Net_teacher= teacher(k_2-k_1,k_4-k_3,100)
gait_Net_teacher.load_state_dict(torch.load("../../datacollection/vicon/subject_1/test/test.pth"))
gait_Net_teacher.to(device)

gait_Net_teacher.eval()

# iterate through batches of test data
with torch.no_grad():
    for i, (data_features_1D, data_features_2D, data_targets) in enumerate(test_loader):
        output_1,output_2,output_3,output = gait_Net_teacher(data_features_1D[:,:,k_1:k_2].to(device).float(),data_features_2D[:,:,:,k_3:k_4].to(device).float())
        if i==0:
          yhat_5=output
          test_target=data_targets

        else:
          yhat_5=torch.cat((yhat_5,output),dim=0)
          test_target=torch.cat((test_target,data_targets),dim=0)




yhat_4 = yhat_5.detach().cpu().numpy()
test_y = test_target.detach().cpu().numpy()
print(yhat_4.shape)

 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,3))
test_y_r=test_y.reshape((test_y.shape[0]*w,3))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/3),3)
test_y_up=test_y_up.reshape(int(len(test_y_up)/3),3)

print(yhat_up.shape,test_y_up.shape)

print(yhat_up.shape)


### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse)

m=rmse

print('\n')
print(m)

print('\n')

print(p)
print('\n')

print(p)

ablation_1=np.hstack([rmse,p])

##################################
first passage through encoder gru
input to encoder_gru, torch.Size([64, 100, 48])
shape of out_1 after lstm1, torch.Size([64, 100, 256])
shape of out_1 dropout1, torch.Size([64, 100, 256])
shape of out_2 after lstm1, torch.Size([64, 100, 128])
shape of out_2 dropout2, torch.Size([64, 100, 128])
shape of out_2 after flatten, torch.Size([64, 12800])
necessary shape for last linear layer = 128000, 600
shape encoder_gru output, torch.Size([64, 300])
shape encoder_gru output after view, torch.Size([64, 100, 3])
##################################
forward pass encoder 1D
shape of input, torch.Size([64, 100, 48])
input_1D_N_1: torch.Size([64, 48, 100])
input_1D_N_1: torch.Size([64, 48, 100])
input_1D_N_1: torch.Size([64, 48, 100])
Shape after transpose: torch.Size([64, 48, 100])
Shape after conv1 relu: torch.Size([64, 64, 100])
Shape after conv2 relu: torch.Size([64, 64, 100])
Shape after BN_2: torch.Size([64, 64, 100])
Shape after pool: torch.Size([64, 64, 5

In [15]:
k_1=36
k_2=48
k_3=6
k_4=8

lr = 0.001
model = student(k_2-k_1,k_4-k_3,100)

gait_Net_student = train_kinematics(device, train_loader, val_loader, lr,num_epoch,model,"../../datacollection/vicon/subject_1/test_student/test_student.pth",k_1, k_2, k_3, k_4)

data_features_1D shape: torch.Size([64, 100, 48])
data_features_2D shape: torch.Size([64, 100, 6, 8])
data_targets shape: torch.Size([64, 100, 3])
##################################
first passage through encoder gru
input to encoder_gru, torch.Size([64, 100, 12])
shape of out_1 after lstm1, torch.Size([64, 100, 256])
shape of out_1 dropout1, torch.Size([64, 100, 256])
shape of out_2 after lstm1, torch.Size([64, 100, 128])
shape of out_2 dropout2, torch.Size([64, 100, 128])
shape of out_2 after flatten, torch.Size([64, 12800])
necessary shape for last linear layer = 128000, 600
shape encoder_gru output, torch.Size([64, 300])
shape encoder_gru output after view, torch.Size([64, 100, 3])
##################################
forward pass encoder 1D
shape of input, torch.Size([64, 100, 12])
input_1D_N_1: torch.Size([64, 12, 100])
input_1D_N_1: torch.Size([64, 12, 100])
input_1D_N_1: torch.Size([64, 12, 100])
Shape after transpose: torch.Size([64, 12, 100])
Shape after conv1 relu: torch.Size([

In [16]:
ait_Net_student= student(k_2-k_1,k_4-k_3,100)
gait_Net_student.load_state_dict(torch.load("../../datacollection/vicon/subject_1/test_student/test_student.pth"))
gait_Net_student.to(device)

gait_Net_student.eval()

# iterate through batches of test data
with torch.no_grad():
    for i, (data_features_1D, data_features_2D, data_targets) in enumerate(test_loader):
        output_1,output_2,output_3,output = gait_Net_student(data_features_1D[:,:,k_1:k_2].to(device).float(),data_features_2D[:,:,:,k_3:k_4].to(device).float())
        if i==0:
          yhat_5=output
          test_target=data_targets

        else:
          yhat_5=torch.cat((yhat_5,output),dim=0)
          test_target=torch.cat((test_target,data_targets),dim=0)




yhat_4 = yhat_5.detach().cpu().numpy()
test_y = test_target.detach().cpu().numpy()
print(yhat_4.shape)

 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,3))
test_y_r=test_y.reshape((test_y.shape[0]*w,3))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/3),3)
test_y_up=test_y_up.reshape(int(len(test_y_up)/3),3)

print(yhat_up.shape,test_y_up.shape)

print(yhat_up.shape)


### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse)

m=rmse

print('\n')
print(m)

print('\n')

print(p)
print('\n')

print(p)

ablation_2=np.hstack([rmse,p])

##################################
first passage through encoder gru
input to encoder_gru, torch.Size([64, 100, 12])
shape of out_1 after lstm1, torch.Size([64, 100, 256])
shape of out_1 dropout1, torch.Size([64, 100, 256])
shape of out_2 after lstm1, torch.Size([64, 100, 128])
shape of out_2 dropout2, torch.Size([64, 100, 128])
shape of out_2 after flatten, torch.Size([64, 12800])
necessary shape for last linear layer = 128000, 600
shape encoder_gru output, torch.Size([64, 300])
shape encoder_gru output after view, torch.Size([64, 100, 3])
##################################
forward pass encoder 1D
shape of input, torch.Size([64, 100, 12])
input_1D_N_1: torch.Size([64, 12, 100])
input_1D_N_1: torch.Size([64, 12, 100])
input_1D_N_1: torch.Size([64, 12, 100])
Shape after transpose: torch.Size([64, 12, 100])
Shape after conv1 relu: torch.Size([64, 64, 100])
Shape after conv2 relu: torch.Size([64, 64, 100])
Shape after BN_2: torch.Size([64, 64, 100])
Shape after pool: torch.Size([64, 64, 5

In [21]:
k_1s=36
k_2s=48
k_3s=6
k_4s=8

lr = 0.001
Student = student(k_2s-k_1s,k_4s-k_3s,100)

k_1t=0
k_2t=48
k_3t=0
k_4t=8

lr = 0.001
Teacher = teacher(k_2t-k_1t,k_4t-k_3t,100)
Teacher.load_state_dict(torch.load("../../datacollection/vicon/subject_1/test/test.pth"))
Teacher.to(device)



student_KD= train_SD(device, alpha, val_loader, train_loader, lr,num_epoch, Student,Teacher, "../../datacollection/vicon/subject_1/test_student/test_student.pth", k_1s, k_2s, k_3s, k_4s, k_1t, k_2t, k_3t, k_4t)

##################################
first passage through encoder gru
input to encoder_gru, torch.Size([64, 100, 12])
shape of out_1 after lstm1, torch.Size([64, 100, 256])
shape of out_1 dropout1, torch.Size([64, 100, 256])
shape of out_2 after lstm1, torch.Size([64, 100, 128])
shape of out_2 dropout2, torch.Size([64, 100, 128])
shape of out_2 after flatten, torch.Size([64, 12800])
necessary shape for last linear layer = 128000, 600
shape encoder_gru output, torch.Size([64, 300])
shape encoder_gru output after view, torch.Size([64, 100, 3])
##################################
forward pass encoder 1D
shape of input, torch.Size([64, 100, 12])
input_1D_N_1: torch.Size([64, 12, 100])
input_1D_N_1: torch.Size([64, 12, 100])
input_1D_N_1: torch.Size([64, 12, 100])
Shape after transpose: torch.Size([64, 12, 100])
Shape after conv1 relu: torch.Size([64, 64, 100])
Shape after conv2 relu: torch.Size([64, 64, 100])
Shape after BN_2: torch.Size([64, 64, 100])
Shape after pool: torch.Size([64, 64, 5

TypeError: unsupported operand type(s) for *: 'student' and 'Tensor'